In [1]:
import pandas as pd
import numpy as np

url = 'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/resstock_dataset_2024.1/resstock_tmy3/metadata_and_annual_results/national/parquet/Baseline/baseline_metadata_and_annual_results.parquet'

df = pd.read_parquet(url)
df = df.head(10)
# df

In [2]:
# df.describe()


In [3]:
# from utils import convert_categorical
def convert_categorical(X: pd.DataFrame):
    category_cols = X.select_dtypes(exclude=np.number).columns.tolist()

    # Convert to Pandas category columns
    for col in category_cols:
        X[col] = X[col].astype('category')
    return X


# df = convert_categorical(df)

in_cols = [col for col in df.columns if col.startswith('in')]
out_cols = [col for col in df.columns if col.startswith('out')]
df['out.electricity.total.energy_consumption.kwh'] = df['out.electricity.total.energy_consumption.kwh'].clip(0,None)
X, y = df[in_cols], df['out.electricity.total.energy_consumption.kwh']

X = convert_categorical(X)
X.describe()

/tmp/ipykernel_27634/2742109806.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = cat_typed
/tmp/ipykernel_27634/2742109806.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = cat_typed
/tmp/ipykernel_27634/2742109806.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,in.sqft,in.simulation_control_run_period_begin_day_of_month,in.simulation_control_run_period_begin_month,in.simulation_control_run_period_calendar_year,in.simulation_control_run_period_end_day_of_month,in.simulation_control_run_period_end_month,in.simulation_control_timestep,in.units_represented,in.weather_file_latitude,in.weather_file_longitude,in.representative_income
count,10.000000,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.000000,10.000000,8.000000
mean,1957.100000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,35.051900,-92.635700,104484.875000
std,905.791054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.984205,15.309826,83046.823353
min,634.000000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,30.000000,-121.600000,22223.000000
25%,1345.500000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,31.442250,-94.306750,42345.750000
50%,1698.000000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,34.826500,-86.733500,86421.500000
75%,2553.250000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,37.891750,-84.317500,149501.000000
max,3310.000000,1.0,1.0,2007.0,31.0,12.0,60.0,1.0,40.783000,-76.420000,267342.000000


In [4]:
## Drop Irrelevant columns and split columns that contain multiple features 

def drop_ignored_columns(X: pd.DataFrame):
    column_plan_df = pd.read_csv('column_plan.csv')
    to_drop = []
    for index, row in column_plan_df.iterrows():
        field = row['field_name']
        keep = row['keep_for_model']
        if keep == "No" and field in X:
            to_drop.append(field)
    X = X.drop(columns=to_drop)
    return X

def str_to_int(text:str):
    if text == "None":
        return 0
    else:
        return int(text)

def split_columns(X:pd.DataFrame):
    # Split possible outcomes by comma, leakage and insulation are two features
    X[['in.duct_leakage','in.duct_insulation']] = X['in.duct_leakage_and_insulation'].str.split(', ',expand=True)
    X = X.drop(columns=['in.duct_leakage_and_insulation'])

    # Columns that are integers with "None" as 0
    X['in.geometry_building_number_units_mf'] = X['in.geometry_building_number_units_mf'].apply(str_to_int)
    X['in.geometry_building_number_units_sfa'] = X['in.geometry_building_number_units_sfa'].apply(str_to_int)

    # Convert string int columns to numeric
    X['in.geometry_building_number_units_mf'] = pd.to_numeric(X['in.geometry_building_number_units_mf'])
    X['in.geometry_building_number_units_sfa'] = pd.to_numeric(X['in.geometry_building_number_units_sfa'])
    X['in.geometry_stories'] = pd.to_numeric(X['in.geometry_stories'])
    X['in.bedrooms'] = pd.to_numeric(X['in.bedrooms'])

    # Split "insulation_wall into two columns - insulation rating, and wall material"
    X['in.wall_type'] = X['in.insulation_wall'].str.split(',').str[0]
    X['in.wall_insulation_rating'] = X['in.insulation_wall'].str.split(', ').str[-1]
    X = X.drop(columns=['in.insulation_wall'])
    
    # Retain only heating type
    X['in.hvac_heating_efficiency'] = X['in.hvac_heating_efficiency'].str.split(',').str[0]

    # Convert compass direction to two-dimensional vector for each feature
    X['in.orientation_northness'] = X['in.orientation']
    X['in.orientation_eastness'] = X['in.orientation']
    X['in.pv_orientation_northness'] = X['in.pv_orientation']
    X['in.pv_orientation_eastness'] = X['in.pv_orientation']
    X = X.drop(columns=['in.orientation', 'in.pv_orientation'])
    return X




def preprocess_columns(X: pd.DataFrame):
    # X = convert_categorical(X)
    X = drop_ignored_columns(X)
    X = split_columns(X)
    X = convert_categorical(X)
    return X

In [5]:
# Add encodings to columns that were just split

custom_binary_cols = ['in.wall_type', 'in.hvac_heating_efficiency']

# Manual Oridinal mappings from columns that were split above
# All other ordinal mapping schemes are in "column_plan.csv"
custom_ordinal_mappings = [
    {
        'col':'in.duct_leakage',
        'mapping':{r'20% Leakage': 20, r'30% Leakage': 30, r'0% Leakage': 0, r'10% Leakage': 10, 'None': -10}
    },
    {
        'col':'in.duct_insulation',
        'mapping':{'Uninsulated': 0, 'R-4': 4, 'R-6': 6, 'R-8': 8, None: -10}},
    {
        'col':'in.wall_insulation_rating',
        'mapping':{'Uninsulated': 0, 'R-11': 11, 'R-15': 15, 'R-19': 19, 'R-7': 7,}
    },
    {
        'col':'in.orientation_northness',
        'mapping':{'North': 4, 'Southwest': 1, 'West': 2, 'Northeast': 3, 'South': 0, 'Northwest': 3, 'East': 2, 'Southeast': 1, "None": 2}
    },
    {
        'col':'in.orientation_eastness',
        'mapping':{'North': 2, 'Southwest': 1, 'West': 0, 'Northeast': 3, 'South': 2, 'Northwest': 1, 'East': 4, 'Southeast': 3, "None": 2}
    },
    {
        'col':'in.pv_orientation_northness',
        'mapping':{'North': 4, 'Southwest': 1, 'West': 2, 'Northeast': 3, 'South': 0, 'Northwest': 3, 'East': 2, 'Southeast': 1, "None": 2}
    },
    {
        'col':'in.pv_orientation_eastness',
        'mapping':{'North': 2, 'Southwest': 1, 'West': 0, 'Northeast': 3, 'South': 2, 'Northwest': 1, 'East': 4, 'Southeast': 3, "None": 2}
    }
]

# X.astype({'in.wall_type':'category', 'in.hvac_heating_efficiency':'category', 'in.duct_leakage':'category', 'in.duct_insulation':'category', 'in.wall_insulation_rating':'category', 'in.orientation_northness':'category', 'in.orientation_eastness':'category', 'in.pv_orientation_northness':'category', 'in.pv_orientation_eastness':'category'})

X = preprocess_columns(X)
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 
# custom_ordinal_mappings.append() 


In [6]:
import json

observed_enums = []
num_enums = []
cols = []
types = []
means = {}
types_dict = {}
for col in X.columns:
    observed_enum = X[col].unique()
    joined = '|'.join(str(v) for v in observed_enum)
    observed_enums.append(joined)
    num_enums.append(len(observed_enum))
    cols.append(col)
    types.append(X[col].dtype)
    if X[col].dtype == 'category':
        means[col] = X[col].mode()[0]
    else:
        means[col] = X[col].mean()
    types_dict[col] = str(X[col].dtype)

X_columns = pd.DataFrame({'column': cols, 'type': types, 'enumerations': observed_enums, 'num_enumerations': num_enums})
X_columns.to_csv('X_columns.csv')

avg_home = {'means': means, 'types': types_dict}

with open("avg_home.json", "w") as outfile: 
    json.dump(avg_home, outfile)

In [7]:
# import json

# means = {}
# types = {}
# for column in X:
#     if df[column].dtype == 'category':
#         means[column] = df[column].mode()[0]
#     else:
#         means[column] = df[column].mean()
#     types[column] = str(df[column].dtype)

# avg_home = {'means': means, 'types': types}

# with open("avg_home.json", "w") as outfile: 
#     json.dump(avg_home, outfile)


In [8]:
'''
Automatically build ordinal mapping format from a string delimited with "|" (csv safe character)
'''
def get_ordinal_mapping(text:str):
    mapping = {}
    split_array = text.split("|")
    for i, item in enumerate(split_array):
        if i % 2 == 1:
            mapping[split_array[i-1]] = int(split_array[i])
    return mapping

In [9]:
## Build Category Encoders from those specified column_plan.csv

import category_encoders as ce

column_plan_df = pd.read_csv('column_plan.csv')

# Binary
binary_fields = column_plan_df[column_plan_df['encoder'] == 'Binary']['field_name'].to_list()
binary_fields += custom_binary_cols
binary_encoder = ce.BinaryEncoder(cols=binary_fields)

# Ordinal
full_mapping = []
for index, row in column_plan_df.iterrows():
    if row['encoder'] != 'Ordinal':
        continue
    mapping_dict = {}
    mapping_dict['col'] = row['field_name']
    mapping_dict['mapping'] = get_ordinal_mapping(row['label_encoder_dict'])
    full_mapping.append(mapping_dict)

full_mapping += custom_ordinal_mappings
ordinal_fields = []
for mapping in full_mapping:
    ordinal_fields.append(mapping['col'])
ordinal_encoder = ce.OrdinalEncoder(cols=ordinal_fields, mapping=full_mapping)

# Catboost
catboost_fields = column_plan_df[column_plan_df['encoder'] == 'CatBoost']['field_name'].to_list()
catboost_encoder = ce.CatBoostEncoder(cols=catboost_fields)


In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pickle

# Build pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_encoder, binary_fields),
        ('ordinal', ordinal_encoder, ordinal_fields),
        ('catboost', catboost_encoder, catboost_fields),
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler())
])

# pipeline.fit(X, y)

# with open('pipeline.pkl', 'wb') as f:
#     pickle.dump(pipeline, f)

In [11]:
from sklearn.model_selection import train_test_split
# X = preprocess_columns(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:


X_train_clean = pipeline.fit(X_train, y_train)

with open('pipeline.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [13]:


with open('pipeline.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)

X_train_clean = loaded_pipeline.transform(X_train)

In [14]:
from xgboost import XGBRegressor


model = XGBRegressor(
    objective = 'reg:squarederror',
    tree_method = 'hist',
    max_depth = 2,
    n_estimators = 50,
    learning_rate = .1,
)

model.fit(X_train_clean, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [15]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GridSearchCV

# estimator = XGBRegressor(
#     objective = 'reg:squarederror',
#     tree_method = 'hist',
#     enable_categorical = True
#     # device="cuda"
# )

# parameters = {
#     'max_depth': range (2),
#     'n_estimators': np.linspace(50, 250, 1, dtype=int),
#     'learning_rate': np.logspace(-2, -.3, 1)
# }

# grid_search = GridSearchCV(
#     estimator=estimator,
#     param_grid=parameters,
#     scoring = 'neg_root_mean_squared_error',
#     n_jobs = -1,
#     cv = 2,
#     verbose=True
# )

# grid_search.fit(X_train, y_train)
# results_df = pd.DataFrame(grid_search.cv_results_)
# model = grid_search.best_estimator_
# results_df

In [16]:


# for index, row in column_plan_df.iterrows():
#     field = row['field_name']
#     if field in df.columns:
#         observed_enum = df[field].unique()
#         actual_enum =  set(observed_enum) #| set(allowable_enum)
#         if len(actual_enum) > 1:
#             joined = '|'.join(str(v) for v in actual_enum)
#             actual_enums.append(joined)
#             num_enums.append(len(actual_enum))
#         else:
#             actual_enums.append(list(actual_enum)[0])
#             num_enums.append(len(actual_enum))
#     else:
#         actual_enums.append("Not observed")
#         num_enums.append("Not observed")

In [17]:
# One-Time function used to get actual enumerations of the baseline data

# column_plan_df = pd.read_csv('column_planning.csv')
# column_plan_df.drop(column_plan_df.columns[column_plan_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# # def create_actual_enumerations(cp_df, x_df):
# actual_enums = []
# num_enums = []
# for index, row in column_plan_df.iterrows():
#     field = row['field_name']
#     if field in df.columns:
#         observed_enum = df[field].unique()
#         actual_enum =  set(observed_enum) #| set(allowable_enum)
#         if len(actual_enum) > 1:
#             joined = '|'.join(str(v) for v in actual_enum)
#             actual_enums.append(joined)
#             num_enums.append(len(actual_enum))
#         else:
#             actual_enums.append(list(actual_enum)[0])
#             num_enums.append(len(actual_enum))
#     else:
#         actual_enums.append("Not observed")
#         num_enums.append("Not observed")



# column_plan_df['actual_enumerations'] = pd.Series(actual_enums)
# column_plan_df['num_enumerations'] = pd.Series(num_enums)
# # column_plan_df.apply(lambda x: create_actual_enumerations(), axis=1)
# # for index, row in column_plan_df.iterrows():
# #     # allowable_enum = row['allowable_enumerations_baseline']
# #     # allowable_enum = allowable_enum.split("|")
# #     field = row['field_name']
# #     if field in df:
# #         observed_enum = df[field].unique()
# #         actual_enum =  set(observed_enum) #| set(allowable_enum)
# #         if len(actual_enum) > 1:
# #             joined = '|'.join(str(v) for v in actual_enum)
# #             column_plan_df.loc[index, 'actual_enumerations'] = joined
# #         else:
# #             column_plan_df.loc[index, 'actual_enumerations'] = actual_enum
# #     else:
# #         column_plan_df.loc[index, 'actual_enumerations'] = "Not observed"

# column_plan_df.to_csv('column_plan.csv', index=False)
# column_plan_df
# df['in.clothes_dryer'].unique()

In [18]:
from sklearn.metrics import mean_squared_log_error

X_test_clean = pipeline.transform(X_test)
preds = model.predict(X_test_clean)
preds = preds.clip(0,None)
rmse = mean_squared_log_error(y_test, preds)

print(f"RMSE of the base model: {rmse:.5f}")

RMSE of the base model: 0.07812


In [ ]:
with open('avg_home.json') as f: 
    data = f.read() 
    
combined_dicts = json.loads(data)
# avg_home_types = avg_home['types']
# avg_home_means = avg_home['means']
avg_home_df = pd.DataFrame(combined_dicts['means'], index=[0])
for column in avg_home_df:
    avg_home_df[column] = avg_home_df[column].astype(combined_dicts['types'][column])
avg_home_df.dtypes
# avg_home = pd.read_json('means.json', orient='records', typ='series')
# avg_home = avg_home.to_frame().T
# avg_home

In [ ]:
var = 'in.ashrae_iecc_climate_zone_2004'
val = '2A'

old_type = str(avg_home_df[var].dtype)
if old_type == 'category' and val not in avg_home_df[var].cat.categories:
    avg_home_df[var] = avg_home_df[var].cat.add_categories([val])

# avg_home_df['in.ashrae_iecc_climate_zone_2004'].cat.add_categories('2A')
avg_home_df.at[0, var] = val
avg_home_df[var] = avg_home_df[var].astype(old_type)
avg_home_df.dtypes


In [ ]:
pred = model.predict(avg_home_df)
pred

In [ ]:
from xgboost import plot_importance
plot_importance(model, max_num_features=10) # top 10 most important features

In [ ]:
booster = model.get_booster()
booster.save_model('model_baseline_alabama.json')

In [7]:
import pandas as pd
data_dict_df = pd.read_csv('X_columns.csv').set_index("column")
data_dict_df = data_dict_df.drop(columns=['Unnamed: 0'])

data_dict_df['enumerations'] = data_dict_df['enumerations'].str.split('|')
# data_dict_df = data_dict_df.replace({"NaN": np.nan})

# Convert the DataFrame to JSON
json_data = data_dict_df.to_json(orient='index')

# Optionally, save the JSON data to a file
with open('docs/data_dictionary.json', 'w') as json_file:
    json_file.write(json_data)

data_dict_df

,type,enumerations,num_enumerations
column,,,
in.ashrae_iecc_climate_zone_2004_2_a_split,category,"[3C, 2A - FL, GA, AL, MS, 3B, 5A, 2A - TX, LA,...",7
in.bathroom_spot_vent_hour,category,"[Hour5, Hour0, Hour20, Hour18, Hour22, Hour16,...",8
in.bedrooms,int64,"[4, 2, 3]",3
in.building_america_climate_zone,category,"[Marine, Hot-Humid, Hot-Dry, Cold, Mixed-Humid]",5
in.ceiling_fan,category,"[Standard Efficiency, None, Standard Efficienc...",3
...,...,...,...
in.wall_insulation_rating,category,"[Uninsulated, R-11, R-15]",3
in.orientation_northness,category,"[North, Southwest, West, Northeast, South, Nor...",6
in.orientation_eastness,category,"[North, Southwest, West, Northeast, South, Nor...",6


In [ ]:
data_dict_df.describe()

In [16]:
# Create county_lookup.json
'''
import pandas as pd
import json
stlt_df = pd.read_csv('spatial_tract_lookup_table.csv')
county_lookup = {}
for state in stlt_df['state_abbreviation'].unique():
    # state_df = stlt_df.loc[stlt_df['state_abbreviation'] == state]
    counties = stlt_df.loc[stlt_df['state_abbreviation'] == state, 'county_name']
    # unique_counties = counties.str.split(', ').str[1].unique().tolist()
    # county_info = {}
    county_lookup[state] = []
    for county in counties.unique().tolist():
        county_name = county.split(', ')[1]
        gisjoin = stlt_df.loc[stlt_df['county_name'] == county, 'nhgis_2010_county_gisjoin'].unique()[0]
        county_info = {'name': county_name, 'gisjoin': gisjoin}
        county_lookup[state].append(county_info)


    # county_info["name"]: 
    # county_lookup[state] = counties.unique().tolist()

# print(counties.str.split(', ').str[1])
with open('county_lookup.json', 'w') as json_file:
    json.dump(county_lookup, json_file)
'''

In [5]:
from epw import epw

e = epw()
e.read('Data/fTMY3/AL/Autauga/fTMY_Alabama_Autauga_B0100101_2060_2079.epw')
epw_df = e.dataframe

print(epw_df.columns.values)


['Year' 'Month' 'Day' 'Hour' 'Minute' 'Data Source and Uncertainty Flags'
 'Dry Bulb Temperature' 'Dew Point Temperature' 'Relative Humidity'
 'Atmospheric Station Pressure' 'Extraterrestrial Horizontal Radiation'
 'Extraterrestrial Direct Normal Radiation'
 'Horizontal Infrared Radiation Intensity' 'Global Horizontal Radiation'
 'Direct Normal Radiation' 'Diffuse Horizontal Radiation'
 'Global Horizontal Illuminance' 'Direct Normal Illuminance'
 'Diffuse Horizontal Illuminance' 'Zenith Luminance' 'Wind Direction'
 'Wind Speed' 'Total Sky Cover'
 'Opaque Sky Cover (used if Horizontal IR Intensity missing)' 'Visibility'
 'Ceiling Height' 'Present Weather Observation' 'Present Weather Codes'
 'Precipitable Water' 'Aerosol Optical Depth' 'Snow Depth'
 'Days Since Last Snowfall' 'Albedo' 'Liquid Precipitation Depth'
 'Liquid Precipitation Quantity']


In [7]:
import pickle
import xgboost as xgb

booster = xgb.Booster()
# model = xgb.XGBRegressor()
booster.load_model('HPC/Jobs/2024-07-27/xgb_pca_model_baseline.json')


with open('HPC/Jobs/2024-07-27/xgb_pca_model_baseline.pkl', 'wb') as f:
    pickle.dump(booster, f)
# pickle.dump(booster, open('HPC/Jobs/2024-07-27/xgb_pca_model_baseline.pkl'))

In [11]:
import pandas as pd
import processingfuncs as pf

url = 'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/resstock_dataset_2024.1/resstock_tmy3/metadata_and_annual_results/national/parquet/Baseline/baseline_metadata_and_annual_results.parquet'

df = pd.read_parquet(url)
# df.head(500).to_parquet('first_500.parquet')
in_cols = [col for col in df.columns if col.startswith('in')]
X, y = df[in_cols], df['out.electricity.total.energy_consumption.kwh']
X = pf.preprocess_columns(X)
X = pf.add_tmy3_data(X)

: 

In [1]:
import pandas as pd
import processingfuncs as pf

df = pd.read_parquet('first_500.parquet')
in_cols = [col for col in df.columns if col.startswith('in')]
X, y = df[in_cols], df['out.electricity.total.energy_consumption.kwh']

# print(X.columns)
print(X['in.duct_leakage_and_insulation'].str.split(', ',expand=True)[1].unique())
X = pf.preprocess_columns(X)
print(X['in.duct_insulation'].unique())
X = pf.add_tmy3_data(X)
nan_count = X.isna().sum()
print(nan_count[nan_count > 0])

['Uninsulated' 'R-4' 'R-6' None 'R-8']
['Uninsulated', 'R-4', 'R-6', 'None', 'R-8']
Categories (5, object): ['None', 'R-4', 'R-6', 'R-8', 'Uninsulated']
Series([], dtype: int64)
